In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, Subset

In [2]:
data = np.load('./mnist_subset_combined.npy')

In [3]:
# Convert the structured array to PyTorch tensors
images = torch.tensor([item['image'] for item in data], dtype=torch.float32)
labels = torch.tensor([item['label'] for item in data], dtype=torch.long)

# Flatten the images for a simple fully connected network
images = images.view(images.shape[0], -1)


# Create a dataset and data loader
dataset = TensorDataset(images, labels)


C:\Users\Sanket\AppData\Local\Temp\ipykernel_24360\2831219842.py:2: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:264.)
  images = torch.tensor([item['image'] for item in data], dtype=torch.float32)


In [4]:
class_indices = {k: [] for k in range(10)}  # Assuming 10 classes (0-9)
for idx, (image, label) in enumerate(dataset):
    class_indices[label.item()].append(idx)

# Step 2: Randomly select 10 samples from each class
labeled_indices = []
for indices in class_indices.values():
    labeled_indices.extend(np.random.choice(indices, 10, replace=False))

# Create a mask for the rest of the data for testing
mask = np.ones(len(dataset), dtype=bool)
mask[labeled_indices] = False
unlabeled_indices = np.arange(len(dataset))[mask]

# Step 3: Create training and testing subsets
labeled_set = Subset(dataset, labeled_indices)
unlabeled_set = Subset(dataset, unlabeled_indices)

In [6]:
# Step 4: Create DataLoaders
labeledLoader = DataLoader(labeled_set, batch_size=10, shuffle=True)
unlabeledLoader = DataLoader(unlabeled_set, batch_size=50, shuffle=False)


In [7]:
class Net(nn.Module):

    def __init__(self):

        super(Net,self).__init__()

        self.fullyConnectedLayer = nn.Sequential(
            nn.Linear(784, 200),
            nn.ReLU(),
            nn.Linear(200,10)
        )

    def forward(self, input):

        output = self.fullyConnectedLayer(input)
        activatedOutput = F.log_softmax(output, dim = 1)

        return activatedOutput
    

In [12]:
def pseudo_label(model, unlabeled_loader, threshold=0.6):
    model.eval()
    pseudo_labeled = []
    for images, _ in unlabeled_loader:
        outputs = model(images)
        prob, max_indices = outputs.max(dim=1)
        mask = prob > threshold
        masked_images = images[mask]
        masked_indices = max_indices[mask]
        pseudo_labeled.append((masked_images, masked_indices))
        
    return torch.cat([x[0] for x in pseudo_labeled]), torch.cat([x[1] for x in pseudo_labeled])

In [9]:
model1 = Net()

In [13]:
import torch.optim as optim

optimizer1 = optim.SGD(model1.parameters(), lr=0.03)
optimizer2 = optim.SGD(model1.parameters(), lr = 0.001)
num_epochs = 100
loss = 0
unsupervised_loss = 0

for epoch in range(num_epochs):
    model1.train()
    
    # Train on labeled data
    for images, labels in labeledLoader:
        optimizer1.zero_grad()
        outputs = model1(images)
        supervised_loss = F.cross_entropy(outputs, labels)
        supervised_loss.backward()
        optimizer1.step()

    print(f"The supervised loss for epoch{epoch} is:{supervised_loss}")

    pseudo_images, pseudo_labels = pseudo_label(model1, unlabeledLoader)
    
    optimizer2.zero_grad()
    outputs = model1(pseudo_images)                    
    pseudo_loss = F.nll_loss(outputs, pseudo_labels) 
    pseudo_loss.backward()
    optimizer2.step()   


    
    print(f"The pseudo loss for epoch{epoch} is:{pseudo_loss}")

The supervised loss for epoch0 is:2.3024277687072754
The pseudo loss for epoch0 is:nan
The supervised loss for epoch1 is:2.303675889968872
The pseudo loss for epoch1 is:nan
The supervised loss for epoch2 is:2.302732229232788
The pseudo loss for epoch2 is:nan
The supervised loss for epoch3 is:2.3059582710266113
The pseudo loss for epoch3 is:nan
The supervised loss for epoch4 is:2.306919574737549
The pseudo loss for epoch4 is:nan
The supervised loss for epoch5 is:2.3041348457336426
The pseudo loss for epoch5 is:nan
The supervised loss for epoch6 is:2.3087477684020996
The pseudo loss for epoch6 is:nan
The supervised loss for epoch7 is:2.304234266281128
The pseudo loss for epoch7 is:nan
The supervised loss for epoch8 is:2.3035664558410645
The pseudo loss for epoch8 is:nan
The supervised loss for epoch9 is:2.304051637649536
The pseudo loss for epoch9 is:nan
The supervised loss for epoch10 is:2.305117130279541
The pseudo loss for epoch10 is:nan
The supervised loss for epoch11 is:2.3043427467

In [11]:
def calculate_accuracy(loader,model):
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in loader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()  

    return 100 * correct / total

train_accuracy = calculate_accuracy(labeledLoader, model1)
test_accuracy = calculate_accuracy(unlabeledLoader, model1)

print(f'Training Accuracy: {train_accuracy}%')
print(f'Testing Accuracy: {test_accuracy}%')

Training Accuracy: 10.0%
Testing Accuracy: 9.818181818181818%


### Entropy minimization for Two Moons Dataset

In [68]:
selected_samples = np.load('./selected_samples.npy')

remaining_samples = np.load('./remaining_dataset.npy')

# Converting the selected samples and remaining samples into PyTorch tensors
selected_samples_tensor = torch.tensor(selected_samples, dtype=torch.float32)
remaining_samples_tensor = torch.tensor(remaining_samples, dtype=torch.float32)

# Extracting features and labels for both datasets
features_selected = selected_samples_tensor[:, :2]
labels_selected = selected_samples_tensor[:, 2].long()  # converting labels to long for classification

features_remaining = remaining_samples_tensor[:, :2]
labels_remaining = remaining_samples_tensor[:, 2].long()

# Creating TensorDatasets
selected_dataset = TensorDataset(features_selected, labels_selected)
remaining_dataset = TensorDataset(features_remaining, labels_remaining)

# Creating DataLoaders
selected_loader = DataLoader(selected_dataset, batch_size=1)  # small batch size for the small dataset
remaining_loader = DataLoader(remaining_dataset, batch_size=10)  # larger batch size for the larger dataset

In [69]:
class TwoMoonsNet(nn.Module):

    def __init__(self):

        super(TwoMoonsNet,self).__init__()

        self.fullyConnectedLayer = nn.Sequential(
            nn.Linear(2, 10),
            nn.ReLU(),
            nn.Linear(10,2)
        )

    def forward(self, input):

        output = self.fullyConnectedLayer(input)
        activatedOutput = F.log_softmax(output, dim = 1)

        return activatedOutput

In [70]:
model2 = TwoMoonsNet()

In [71]:
import torch.optim as optim

optimizer1 = optim.SGD(model2.parameters(), lr=0.03)
optimizer2 = optim.SGD(model2.parameters(), lr = 0.01)
num_epochs = 10
loss = 0
unsupervised_loss = 0

for epoch in range(num_epochs):
    model2.train()
    
    # Train on labeled data
    for images, labels in selected_loader:
        optimizer1.zero_grad()
        outputs = model2(images)
        loss = F.cross_entropy(outputs, labels)
        loss.backward()
        optimizer1.step()

    print(f"The supervised loss for epoch{epoch} is:{loss}")
    
    # Train on unlabeled data
    for images, labels in remaining_loader:
        optimizer2.zero_grad()
        outputs = model2(images)
        unsupervised_loss = entropy_loss(outputs)
        unsupervised_loss.backward()
        optimizer2.step()

    
    print(f"The un-supervised loss for epoch{epoch} is:{unsupervised_loss}")

The supervised loss for epoch0 is:0.48841455578804016
The un-supervised loss for epoch0 is:0.5801041126251221
The supervised loss for epoch1 is:0.42923659086227417
The un-supervised loss for epoch1 is:0.5446867942810059
The supervised loss for epoch2 is:0.38170090317726135
The un-supervised loss for epoch2 is:0.5164116621017456
The supervised loss for epoch3 is:0.34158214926719666
The un-supervised loss for epoch3 is:0.4931500256061554
The supervised loss for epoch4 is:0.3071553409099579
The un-supervised loss for epoch4 is:0.4764765202999115
The supervised loss for epoch5 is:0.2772395610809326
The un-supervised loss for epoch5 is:0.46371200680732727
The supervised loss for epoch6 is:0.2506945729255676
The un-supervised loss for epoch6 is:0.45309799909591675
The supervised loss for epoch7 is:0.22713176906108856
The un-supervised loss for epoch7 is:0.4448125660419464
The supervised loss for epoch8 is:0.20651240646839142
The un-supervised loss for epoch8 is:0.4384482800960541
The supervi

In [72]:
train_accuracy = calculate_accuracy(selected_loader, model2)
test_accuracy = calculate_accuracy(remaining_loader, model2)

print(f'Training Accuracy: {train_accuracy}%')
print(f'Testing Accuracy: {test_accuracy}%')

Training Accuracy: 100.0%
Testing Accuracy: 82.97872340425532%
